# Extracts Device Distances
Extract the device distances and compares them with the genetic distances


In [1]:
import sys
sys.path.append('/home/minigonche/covid_contact_graphs')

import pandas as pd
import functions.utils as utils
from google.cloud import bigquery
from matplotlib import pyplot as plt
from scipy import stats
from datetime import datetime, timedelta
client = bigquery.Client(location="US")
import networkx as nx
import numpy as np
from datetime import timedelta
pd.set_option('display.max_rows', 50)

from os.path import exists



# Samples and Devices 

In [2]:
sql = """
    SELECT * from `grafos-alcaldia-bogota.samples_analysis.samples_and_devices_ids`
"""

df_dev = utils.run_simple_query(client=client, query=sql)


In [4]:
# Filters
df_dev = df_dev[df_dev.date >= pd.to_datetime("2021-07-31")].dropna(subset= ['identifier'])
print(f"Unique Samples: {len(df_dev.sample_id.unique())}")
df_dev

Unique Samples: 48


/home/minigonche/env/lib/python3.9/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


,sample_id,date,address,lat,lon,identifier,distance,total_days_interventions,total_hours_interventions,total_days_control,total_hours_control,total_days_interventions_in_schedule,total_hours_interventions_in_schedule,is_intervention,is_control,is_intervention_in_schedule
0,80209534,2021-07-31,KR 78 52 B 60,4.677677,-74.111894,b211cde2-051e-4cae-b35e-57a8a7e3e40e,15.937359,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False
1,80209534,2021-07-31,KR 78 52 B 60,4.677677,-74.111894,bf8cea60-01d7-4b0d-a5c6-9118aa085404,16.141162,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False
2,80209534,2021-07-31,KR 78 52 B 60,4.677677,-74.111894,dcd013b7-6fa0-445d-9115-3cf0705d22fd,17.487361,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False
3,80209534,2021-07-31,KR 78 52 B 60,4.677677,-74.111894,930f3be5-5333-4e32-8485-dba49939ff1e,18.543568,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False
4,80209534,2021-07-31,KR 78 52 B 60,4.677677,-74.111894,8e8f2d8d-f8db-41b5-84c0-a3b85a349b47,19.949967,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,80209534,2021-07-31,KR 78 52 B 60,4.677677,-74.111894,72f56dc7-a495-4bba-b9a0-8ec7040f4443,28.360119,11.0,45.0,0.0,0.0,5.0,15.0,True,False,True
388,82809549,2021-08-27,TV 78H BIS 48A 24 SUR,4.613934,-74.166305,deda5ebf-10da-4bed-90ba-eb00fac8a5a9,15.198738,11.0,63.0,0.0,0.0,2.0,6.0,True,False,True
389,82809549,2021-08-27,TV 78H BIS 48A 24 SUR,4.613934,-74.166305,e12eca27-48cf-443d-bd8c-3f3b32e85567,15.787530,4.0,12.0,0.0,0.0,1.0,3.0,True,False,True
390,82809549,2021-08-27,TV 78H BIS 48A 24 SUR,4.613934,-74.166305,4ccfa3f7-cc25-4e55-b6d6-3ad5665d3efb,18.041276,5.0,18.0,0.0,0.0,1.0,3.0,True,False,True


In [4]:
min_date = df_dev.date.min().strftime("%Y-%m-%d")
max_date = df_dev.date.max().strftime("%Y-%m-%d")
print(f"{min_date} {max_date}")


2021-07-31 2022-01-05


## Builds Graphs
For each date computes the graph of the previous 14 days and computes all the distances from the sample of that day (Samples A), to the samples of the previous 14 days (Sample B). 
Notice that Sample A is included in Sample B, but the distances between elements of Sample B are not computed, to discard future contacts.

In [5]:
num_days = 14
extension = 'xz'
save = False
cache = True
table_id = "grafos-alcaldia-bogota.edgelists_cities.colombia_bogota"
#table_id = "grafos-alcaldia-bogota.edgelists_bogota_localities.colombia_bogota_localidad_usaquen"

In [6]:
# Iterates over all the dates
dates = df_dev[['date']].drop_duplicates().sort_values('date',ascending = False)['date']


j = 0
for d in dates:

    print(f"{d} ({j+1} of {dates.shape[0]})")
    d_str = d.strftime("%Y-%m-%d")
    
    j += 1

    df_selected_dev = df_dev[(df_dev.date <= d)&(df_dev.date >= d - timedelta(days=num_days))]

    all_device_nodes = df_selected_dev.identifier.drop_duplicates().values


    # Builds the distance df
    df1 = df_selected_dev.loc[df_selected_dev.date == d, ['sample_id']].drop_duplicates().rename(columns = {'sample_id': 'sample_id_1'})
    df2 = df_selected_dev[['sample_id']].drop_duplicates().rename(columns = {'sample_id': 'sample_id_2'})
    df1['key'] = 1
    df2['key'] = 1
    df_distances = df1.merge(df2, on = "key")
    df_distances = df_distances.drop('key', axis = 1)

    # Adds devices
    df_distances = df_distances.merge(df_selected_dev[['sample_id','identifier']].rename(columns = {'sample_id': 'sample_id_1','identifier': 'identifier_1'}))
    df_distances = df_distances.merge(df_selected_dev[['sample_id','identifier']].rename(columns = {'sample_id': 'sample_id_2','identifier': 'identifier_2'}))  
    df_distances['distance'] = np.inf
    df_distances.loc[df_distances.sample_id_1== df_distances.sample_id_2, 'distance'] = 0
    df_distances.loc[df_distances.identifier_1 == df_distances.identifier_2, 'distance'] = 0


    sample_distances = df_distances[['sample_id_1','sample_id_2','distance']].groupby(['sample_id_1','sample_id_2']).min().reset_index()
    
    if sample_distances['distance'].max() == 0:
        df_distances.to_csv(f'temp/{table_id.split(".")[-1]}_{d_str}.csv', index = False)
        print("   All samples have devices in common")
        continue
    
    
    #if G is not None and cache:
    sql = f""" 
        SELECT id1, id2, COUNT(*) as weight 
        FROM  `{table_id}` as con
        WHERE date <= "{d_str}"
        and date >= DATE_SUB("{d_str}", INTERVAL {num_days} DAY)
        GROUP BY id1, id2
    """

    print("   Gets Edges")

    file_name = f"temp/edges_{d_str}_{num_days}.{extension}"

    # Checks for cache
    if exists(file_name):
        df_edges = pd.read_pickle(file_name)
    else:
        print("      No file found. Loads from query")
        df_edges = utils.run_simple_query(client, sql, True)
        print(f"         Saving. Size: {df_edges.shape[0]} rows")
        if save:
            df_edges.to_pickle(file_name)
        
    print('   Builds Graph')
    nodes = list(set(df_edges.id1.values).union(df_edges.id2.values).union(all_device_nodes))

    # Builds Graph
    G = nx.Graph()
    G.add_nodes_from(nodes)
    G.add_edges_from([(row.id1, row.id2) for _, row in df_edges.iterrows()])

    # Computes distances
    df_device_pairs = df_distances.loc[ df_distances.distance > 0, ['identifier_1','identifier_2']].drop_duplicates()


    
    i = 0
    prog = int(max(np.round(df_device_pairs.shape[0]/10),1))
    for _, row in df_device_pairs.iterrows():

        if i % prog == 0:
            print(f"      {i+1} of {df_device_pairs.shape[0]}")

        i+=1
        try:
            id1 = row.identifier_1
            id2 = row.identifier_2
            dist = nx.shortest_path_length(G, source=id1, target=id2)
        
            df_distances.loc[(df_distances.identifier_1 == id1)&(df_distances.identifier_2 == id2),'distance'] = dist
            
        except nx.NetworkXNoPath:
            continue

    # Saves progress
    df_distances.to_csv(f'temp/{table_id.split(".")[-1]}_{d_str}.csv', index = False)

print()
print("----------------")
print('Done')
    


2022-01-05 (1 of 33)
   Gets Edges
      No file found. Loads from query
         Saving. Size: 11443290 rows
   Builds Graph
      1 of 1
2021-12-31 (2 of 33)
   Gets Edges
      No file found. Loads from query
         Saving. Size: 11421612 rows
   Builds Graph
      1 of 3
      2 of 3
      3 of 3
2021-12-20 (3 of 33)
   Gets Edges
      No file found. Loads from query
         Saving. Size: 12771721 rows
   Builds Graph
      1 of 2
      2 of 2
2021-12-18 (4 of 33)
   All samples have devices in common
2021-12-01 (5 of 33)
   Gets Edges
      No file found. Loads from query
         Saving. Size: 1459803 rows
   Builds Graph
      1 of 24
      3 of 24
      5 of 24
      7 of 24
      9 of 24
      11 of 24
      13 of 24
      15 of 24
      17 of 24
      19 of 24
      21 of 24
      23 of 24
2021-11-30 (6 of 33)
   Gets Edges
      No file found. Loads from query
         Saving. Size: 1414952 rows
   Builds Graph
      1 of 31
      4 of 31
      7 of 31
      10 of 31
   

In [7]:
all_distances = []
for d in dates:

    d_str = d.strftime("%Y-%m-%d")
    all_distances.append(pd.read_csv(f'temp/{table_id.split(".")[-1]}_{d_str}.csv'))

final_device_distances = pd.concat(all_distances)
final_sample_distances = final_device_distances[['sample_id_1','sample_id_2','distance']].groupby(['sample_id_1','sample_id_2']).min().reset_index()


,sample_id_1,sample_id_2,distance
0,10509598,10509598,0.0
1,10509598,123109559,inf
2,80209534,80209534,0.0
3,80209554,80209534,inf
4,80209554,80209554,0.0
...,...,...,...
315,122009541,121809505,inf
316,122009541,122009541,0.0
317,123109559,121809505,inf
318,123109559,122009541,inf


In [8]:
final_sample_distances.distance.value_counts()

inf    258
0.0     56
4.0      4
7.0      1
2.0      1
Name: distance, dtype: int64

In [9]:
final_sample_distances.to_csv('sample_distaces.csv', index=False)